First, we upload the TESS Sectors 1 and 2 Target Lists (in this repo, and obtainable [here](https://tess.mit.edu/observations/sector-1/) and [here](https://tess.mit.edu/observations/sector-2/)) to ESA's [Gaia Archive](https://gea.esac.esa.int/archive/). Then we run the following ADQL query:

```SQL
SELECT gaia.*, tess.*, gdist.*, distance(
  POINT('ICRS', tess.ra, tess.dec),
  POINT('ICRS', gaia.ra, gaia.dec)) AS dist
FROM gaiadr2.gaia_source AS gaia, USERNAME.tess_sectorN AS tess
JOIN external.gaiadr2_geometric_distance as gdist ON gdist.source_id = gaia.source_id
WHERE 1=CONTAINS(
  POINT('ICRS', tess.ra, tess.dec),
  CIRCLE('ICRS', gaia.ra, gaia.dec, DISTANCE)
)
```

where DISTANCE is 1 or 5 arcseconds, converted to decimal degrees.

Reading in the data is as simple as

In [ ]:
import pandas as pd

sector1 = pd.read_csv(TABLE NAME)
sector2 = pd.read_csv(TABLE NAME)

data = pd.concat([sector1,sector2])

We have all columns available from Gaia and the TESS target lists (plus the distance on the sky between the TESS and Gaia source), as well as the geometric distances from Bailor-Jones et al. 2018

In [ ]:
data.columns

We can have some fun with the data. Let's estimate the absolute Gaia magnitude $M_G$, as well as deredden the Gaia $G_{BP}-G_{RP}$ color using coefficients from Malhan, Ibata & Martin (2018a)

In [ ]:
data['M_G'] = data['phot_g_mean_mag'] - 5 * np.log10(data['r_est']) + 5 - data['a_g_val']

#Coefficients from Malhan, Ibata & Martin (2018a), ratio of A_X/A_V
ag_av = 0.85926
ab_av = 1.06794
ar_av = 0.65199

data['E(BP-RP)'] = (data['a_g_val']/ag_av)*(ab_av - ar_av)
data['(BP_RP)_0'] = data['phot_bp_mean_mag'] - data['phot_rp_mean_mag'] - data['E(BP-RP)']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.scatter(data['(BP_RP)_0'],data['M_G'])
plt.gca().invert_yaxis()
plt.xlabel(r'$G_{BP}-G_{RP}$')
plt.ylabel(r'$M_G$')